<a href="https://colab.research.google.com/github/lukkychan/resolution_changer/blob/main/new2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

!rm -rf frames
!rm -rf frames/720p
!rm -rf frames/360p
!rm -rf scaled
!rm -rf predicted
!rm -rf Pre_orginal
!rm -rf dataset
!rm -rf dataset/720p
!rm -rf dataset/360s
!rm -rf checkpoints
!rm -rf temp_model
!rm -rf temp_scaled


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!mkdir -p frames/720p
!mkdir -p frames/360p
!mkdir scaled
!mkdir predicted
!mkdir dataset
!mkdir -p dataset/720p
!mkdir -p dataset/360p
!mkdir checkpoints

In [4]:
import cv2

# Specify the path to the video file
video_path = "/content/drive/MyDrive/model/1080p.mp4"

# Specify the output folder to save the frames
output_folder = "/content/frames/360p/"

# Open the video file
video = cv2.VideoCapture(video_path)

# Get the total number of frames in the video
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
print(total_frames)

# Calculate the frame interval to evenly sample the frames
frame_interval = max(total_frames//125, 1)

# Initialize a counter to keep track of the extracted frames
frame_count_1 = 0

# Loop through the frames and extract the desired number of frames
while frame_count_1 <= 125 :
    # Read the current frame
    ret, frame = video.read()

    # Check if the frame was successfully read
    if not ret:
        break

    # Save the frame as an image file
    frame_path = f"{output_folder}{frame_count_1}.jpg"
    cv2.imwrite(frame_path, frame)

    # Increment the frame count
    frame_count_1 += 1

    # Move to the next frame based on the frame interval
    video.set(cv2.CAP_PROP_POS_FRAMES, frame_count_1 * frame_interval)

# Release the video capture object
video.release()
print(frame_count_1)
print("done.")


2987
126
done.


In [5]:
import cv2

# Specify the path to the video file
video_path = "/content/drive/MyDrive/model/720p.mp4"

# Specify the output folder to save the frames
output_folder = "/content/frames/720p/"

# Open the video file
video = cv2.VideoCapture(video_path)

# Get the total number of frames in the video
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
print(total_frames)

# Calculate the frame interval to evenly sample the frames
frame_interval = max(total_frames// 125 , 1)

# Initialize a counter to keep track of the extracted frames
frame_count_2 = 0

# Loop through the frames and extract the desired number of frames
while frame_count_2 <= 125 :
    # Read the current frame
    ret, frame = video.read()

    # Check if the frame was successfully read
    if not ret:
        break

    # Save the frame as an image file
    frame_path = f"{output_folder}{frame_count_2}.jpg"
    cv2.imwrite(frame_path, frame)

    # Increment the frame count
    frame_count_2 += 1

    # Move to the next frame based on the frame interval
    video.set(cv2.CAP_PROP_POS_FRAMES, frame_count_2 * frame_interval)

# Release the video capture object
video.release()
print(frame_count_2)
print("done.")


2986
126
done.


In [6]:
import gc
gc.collect()

0

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
import cv2
import torchvision

# Check if a GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the original 1080p image
original_image = cv2.imread('/content/frames/360p/110.jpg')

# Calculate the scale factor
original_width = original_image.shape[1]
target_width = 1920
scale_factor = target_width / original_width

# Downscale the image using OpenCV's resize function
downscaled_image = cv2.resize(original_image, (int(original_width * scale_factor), int(original_image.shape[0] * scale_factor)))

# Convert the images to tensors and move them to the GPU
downscaled_image = transforms.ToTensor()(downscaled_image).unsqueeze(0).to(device)
original_image = transforms.ToTensor()(original_image).unsqueeze(0).to(device)

# Create a CNN model for upscaling
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU()

    def forward(self, x):
        residual = x
        out = self.relu(self.conv1(x))
        out = self.conv2(out)
        out += residual
        out = self.relu(out)
        return out

class UpscaleModel(nn.Module):
    def __init__(self):
        super(UpscaleModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.residual_blocks = nn.Sequential(
            ResidualBlock(64),
            ResidualBlock(64),
            ResidualBlock(64),
            ResidualBlock(64),
            ResidualBlock(64)
        )
        self.conv2 = nn.Conv2d(64, 3, kernel_size=3, padding=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.conv1(x)
        out = self.residual_blocks(out)
        out = self.conv2(out)
        out = self.sigmoid(out + x)  # Add skip connection to preserve details
        return out

# Create the model and move it to the GPU
model = UpscaleModel().to(device)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create a DataLoader for the training data
train_dataset = TensorDataset(downscaled_image, original_image)
train_loader = DataLoader(train_dataset, batch_size=1)

# Train the model
model.train()
for epoch in range(10):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}: Loss = {running_loss/len(train_loader):.4f}")

# Save the trained model
torch.save(model.state_dict(), 'upscale_model.pth')

import gc
gc.collect()


Epoch 1: Loss = 0.1828
Epoch 2: Loss = 0.1304
Epoch 3: Loss = 0.0576
Epoch 4: Loss = 0.0251
Epoch 5: Loss = 0.0283
Epoch 6: Loss = 0.0283
Epoch 7: Loss = 0.0283
Epoch 8: Loss = 0.0283
Epoch 9: Loss = 0.0283
Epoch 10: Loss = 0.0283


0

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
import cv2
from skimage.metrics import structural_similarity as ssim


model = UpscaleModel()
model.load_state_dict(torch.load('upscale_model.pth'))
# Use the trained model to upscale the downscaled image
model.eval()
with torch.no_grad():
    upscaled_image = model(downscaled_image)

# Convert the upscaled image to a numpy array and move it to the CPU
upscaled_image = upscaled_image.squeeze(0).cpu().numpy()

# Upscale the image using OpenCV's resize function
upscaled_image = cv2.resize(upscaled_image.transpose(1, 2, 0), (original_width, original_image.shape[2]))

# Convert the upscaled image to a tensor and move it to the GPU
upscaled_image = transforms.ToTensor()(upscaled_image).unsqueeze(0).to(device)

# Calculate the SSIM and PSNR
ssim_score = ssim(original_image.squeeze(0).permute(1, 2, 0).cpu().numpy(), upscaled_image.squeeze(0).permute(1, 2, 0).cpu().numpy(), multichannel=True)
mse = torch.mean((original_image - upscaled_image)**2)
psnr = 10 * torch.log10(1.0 / mse)

print(f"SSIM: {ssim_score:.4f}")
print(f"PSNR: {psnr.item():.2f} dB")

<ipython-input-10-b0d2cb2093b2>:27: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_score = ssim(original_image.squeeze(0).permute(1, 2, 0).cpu().numpy(), upscaled_image.squeeze(0).permute(1, 2, 0).cpu().numpy(), multichannel=True)


SSIM: 0.3818
PSNR: 15.50 dB


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
import cv2

# Check if a GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the original 720p image
original_image = cv2.imread('/content/frames/360p/110.jpg')

# Calculate the scale factor
original_width = original_image.shape[1]
target_width = 1920
scale_factor = target_width / original_width

# Downscale the image using OpenCV's resize function
downscaled_image = cv2.resize(original_image, (int(original_width * scale_factor), int(original_image.shape[0] * scale_factor)))

# Convert the images to tensors and move them to the GPU
downscaled_image = transforms.ToTensor()(downscaled_image).unsqueeze(0).to(device)
original_image = transforms.ToTensor()(original_image).unsqueeze(0).to(device)

# Create a CNN model for upscaling

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU()

    def forward(self, x):
        residual = x
        out = self.relu(self.conv1(x))
        out = self.conv2(out)
        out += residual
        out = self.relu(out)
        return out

class UpscaleModel(nn.Module):
    def __init__(self):
        super(UpscaleModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.residual_blocks = nn.Sequential(
            ResidualBlock(64),
            ResidualBlock(64),
            ResidualBlock(64),
            ResidualBlock(64),
            ResidualBlock(64)
        )
        self.conv2 = nn.Conv2d(64, 3, kernel_size=3, padding=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.conv1(x)
        out = self.residual_blocks(out)
        out = self.conv2(out)
        out = self.sigmoid(out + x)  # Add skip connection to preserve details
        return out

# Create the model and move it to the GPU
model = UpscaleModel().to(device)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create a DataLoader for the training data
train_dataset = TensorDataset(downscaled_image, original_image)
train_loader = DataLoader(train_dataset, batch_size=1)

# Train the model
model.train()
for epoch in range(10):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}: Loss = {running_loss/len(train_loader):.4f}")

# Save the trained model
torch.save(model.state_dict(), 'upscale_model.pth')
